In [17]:
#>1#(70337463):(70337463)
import matplotlib.pyplot as plt
#<1#(70337463)~%(1904052177) #>1#(64727859):(64727859)
from tensorflow.keras.losses import sparse_categorical_crossentropy
#<1#(64727859)~%(-1952786418) #>1#(17199287):(17199287)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
#<1#(17199287)~%(533338258) #>1#(60489680):(60489680)
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
#<1#(60489680)~%(696278483)
# Leave for imports

ModuleNotFoundError: No module named 'tensorflow'

Run the following !pip3 cells to install scikit-plot and tensorflow which are needed for this sandbox.

In [ ]:
!pip3 install scikit-plot

In [ ]:
!pip3 install tensorflow

We're a little briefer in the sandbox since it builds on the Neural Network Classifer sandbox.

# Neural Network Regression


This time, we will build from the Neural Network Classifier sandbox. We want you to see that you don't have to rework the notebook into a regression model, as opposed to a classifer, you only need to specify what you want to predict. Arctic fox will know that the data being predicting matches a regression model. It will apply this difference through out the code:
- the predicted column won't be one-hot encoded
- the final layer of the model won't be a softmax
- accuracy won't be included in the compilation or visualization
- the confusion matrix is replaced with a graph of the actual vs predicted 

And others. Check out the sand box and, as always, feel free to make changes. 

Again, the wine column lacks headers, predicting column 7 will predict the Flavanoids in each wine. What are those? We don't really know... But we do know its a regression column! 

In [ ]:
#[Data wine.csv --predict 7]#@60489680 #>1#(60489680):(60489680)
wine = pd.read_csv('wine.csv')
wineAllFeatures = wine.copy()

column0ToNumber = {
    1: 0, 2: 1, 3: 2
}

wineAllFeatures.iloc[:,0] = wineAllFeatures.iloc[:,0].apply(lambda cell : column0ToNumber[cell])

wineAllLabels = wineAllFeatures.iloc[:,7]
wineAllFeatures.drop(wineAllFeatures.columns[7], axis=1, inplace=True)
wineAllFeatures = np.array(wineAllFeatures)

wineTrainingFeatures, wineTestFeatures, wineTrainingLabels, wineTestLabels = train_test_split(wineAllFeatures, wineAllLabels, test_size=0.2)
#<1#(60489680)~%(-1397202326)

Play around with different size networks. When we were testing the wine dataset, a smaller network seemed to work better. 

In [ ]:
#[NN --denseLayers 8 --denseStart 24 --denseEnd 8 --dropOutRatio 1:4]#@17199287 #>1#(17199287):(17199287)
# Create the model
model = Sequential()

model.add(Dense(24, activation='relu', input_shape=(13,)))
model.add(Dense(19, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(11, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))
#<1#(17199287)~%(-1745109498)

In [ ]:
#>1#(17199287):(17199287)
# Compile the model
model.compile(
    loss='mean_squared_error',
    optimizer=Adam()
    )
#<1#(17199287)~%(-672719537)

In [ ]:
#>1#(17199287):(17199287)
model.summary()
#<1#(17199287)~%(1975814818)

Don't be afraid to train quite a bit. We ended up training for around 1500 total epochs. 

In [ ]:

#[Train --epochs 100]#@64727859 #>1#(64727859):(64727859)
# Fit data to model
history = model.fit(
    wineTrainingFeatures,
    wineTrainingLabels,
    batch_size=1,
    epochs=100,
    verbose=1,
    validation_split=0.2
)
#<1#(64727859)~%(-977557149)

In [ ]:
#>1#(64727859):(64727859)
# Generate generalization metrics
score = model.evaluate(wineTestFeatures, wineTestLabels, verbose=0, batch_size=1)
print(f'Test loss: {score}')
#<1#(64727859)~%(-1280751450)

View the loss and predictions. With the --predictions item, Arctic Fox will graph the predictions and labels so you can visually see how close the two are. 

In [ ]:
#[Visualize --loss --predictions]#@70337463 #>1#(70337463):(70337463)
#***Plot history: Loss
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='val')
plt.title('Train and Validation Loss History')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
#<1#(70337463)~%(-52552)

In [ ]:
#>1#(70337463):(70337463)
#***Automated for item: --predict. View predicted values vs actual label
prediction = model.predict(wineTestFeatures)
labels = [label for label in wineTestLabels]
nonArrayPrediction = [value[0] for value in prediction]
zipped = zip(labels, nonArrayPrediction)
sortedList = sorted(zipped, key=lambda value: value[0])
sortedLabels = [value[0] for value in sortedList]
sortedPredictions = [value[1] for value in sortedList]

xAxis = np.arange(0, len(prediction), 1)
plt.scatter(xAxis, sortedLabels, label='actual', color='lawngreen')
plt.scatter(xAxis, sortedPredictions, label='prediction', color='blueviolet')
plt.legend(loc='upper left')
plt.xlabel('index')
#<1#(70337463)~%(2121028170)

There you have it! You can train the network more to improve its performance, try changing the model archiecture, etc. And then, check out the next sandbox!